# Chapter 7: Finding optimal restrictions via exhaustive search

This notebook contains the Cournot and Braess experiments presented in Chapter 7 of the thesis. 

In the experiments, agents act rationally by choosing the best response to their opponent's actions. It is the task of the governance to restrict the action spaces such that the social welfare (i.e., the sum of agent utilities) at the Nash Equilibrium is maximized. This number is called MESU (Minimum Equilibrium Social Utility) and represents the main KPI of the experiments.

We show, for a number of runs with different parameter values, the MESU with and without restrictions, as well as its relative improvement, the degree of restriction and the number of oracle calls.

## Setup

### Imports

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tqdm import tqdm

from src.solver import IntervalUnionRestrictionSolver
from src.utility import QuadraticTwoPlayerUtility
from src.nfg import GovernedNormalFormGame
from src.utils import IntervalUnion, degree_of_restriction, relative_improvement
from src.equilibria import worst_hill_climbing_nash_equilibrium

## Experiments

### Parameterized Cournot Game (CG)

#### Simulation

In [ ]:
results = []
epsilon, decimals = 0.1, 3
solver = IntervalUnionRestrictionSolver(epsilon=epsilon)
lambda_min, lambda_max = 10.0, 200.0
lambdas = list(np.round(np.arange(lambda_min, lambda_max, 1.0), decimals=decimals))

print(f'Solving {len(lambdas)} Cournot games...')
for i, lambda_ in tqdm(enumerate(lambdas), total=len(lambdas)):
    u_1 = QuadraticTwoPlayerUtility(0, [-1.0, 0.0, -1.0, lambda_, 0.0, 0.0])
    u_2 = QuadraticTwoPlayerUtility(1, [0.0, -1.0, -1.0, 0.0, lambda_, 0.0])

    a = IntervalUnion([(0.0, lambda_)])
    g = GovernedNormalFormGame(a, [u_1, u_2], u_1 + u_2)

    results.append(solver.solve(g, nash_equilibrium_oracle=worst_hill_climbing_nash_equilibrium))

#### Visualization

In [ ]:
if not os.path.exists('results/cournot'):
    os.makedirs('results/cournot')

In [ ]:
X = lambdas
fig, ax1 = plt.subplots(figsize=(8, 4))
plt.xlabel('$\\lambda$')

ax1.set_ylabel('MESU')

Y = [result.initial_social_utility for result in results]
ax1.plot(X, Y, label='Unrestricted MESU')

Y = [result.optimal_social_utility for result in results]
ax1.plot(X, Y, label='Restricted MESU')

ax2 = ax1.twinx()
ax2.set_ylabel('$\\Delta(R^*)$')
ax2.set_ylim([0.0, 30.0])
ax2.yaxis.set_major_formatter(PercentFormatter())
Y = [100.0 * relative_improvement(result) for result in results]
ax2.plot(X, Y, color='g', label='Relative improvement')

fig.legend()

# Save graph
fig.savefig(f'results/cournot/mesu.pdf', dpi=300, bbox_inches='tight')

In [ ]:
X = lambdas
fig, ax1 = plt.subplots(figsize=(8, 4))

plt.xlabel('$\\lambda$')
ax1.set_ylabel('$\\mathfrak{r}(R^*)$')
ax1.set_ylim([20.0, 30.0])
ax1.yaxis.set_major_formatter(PercentFormatter(decimals=0))
Y = [100 * degree_of_restriction(result) for result in results]
ax1.plot(X, Y, label='Degree of restriction')

ax2 = ax1.twinx()
ax2.set_ylabel('# oracle calls')
Y = [result.info['number_of_oracle_calls'] for result in results]
ax2.plot(X, Y, color='g', label='Number of oracle calls')

fig.legend()

# Save graph
fig.savefig(f'results/cournot/degree.pdf', dpi=300, bbox_inches='tight')

### Parameterized Continuous Braess Paradox (BP)

#### Simulation

In [ ]:
results = []
epsilon, decimals = 0.0001, 5
solver = IntervalUnionRestrictionSolver(epsilon=epsilon)
b_min, b_max, b_step = 4.0, 18.0, 0.1
bs = list(np.round(np.arange(b_min, b_max, b_step), decimals=decimals))
params = [(0.0, b, 4.0, 0.0) for b in bs]

print(f'Solving {len(params)} Braess games...')
for i, [a, b, c, d] in tqdm(enumerate(params), total=len(params)):
    u_1 = QuadraticTwoPlayerUtility(0, [-a - c, 0.0, 0.0, 2*a + b - c - 1, -c, 4*c + d + 1])
    u_2 = QuadraticTwoPlayerUtility(1, [0.0, -a - c, 0.0, -c, 2*a + b - c - 1, 4*c + d + 1])

    a = IntervalUnion([(0.0, 1.0)])
    g = GovernedNormalFormGame(a, [u_1, u_2], u_1 + u_2)

    results.append(solver.solve(g, nash_equilibrium_oracle=worst_hill_climbing_nash_equilibrium))

#### Visualization

In [ ]:
if not os.path.exists('results/braess'):
    os.makedirs('results/braess')

In [ ]:
X = np.array([b for a, b, c, d in params])
fig, ax1 = plt.subplots(figsize=(8, 4))
plt.xlabel('$b$')

ax1.set_ylabel('MESU')

Y = [result.initial_social_utility for result in results]
ax1.plot(X, Y, label='Unrestricted MESU')

Y = [result.optimal_social_utility for result in results]
ax1.plot(X, Y, label='Restricted MESU')

ax2 = ax1.twinx()
ax2.set_ylabel('$\\Delta(R^*)$')
ax2.yaxis.set_major_formatter(PercentFormatter())
Y = [100.0 * relative_improvement(result) for result in results]
ax2.plot(X, Y, color='g', label='Relative improvement')

fig.legend()

# Save graph
fig.savefig(f'results/braess/mesu.pdf', dpi=300, bbox_inches='tight')

In [ ]:
X = np.array([b for a, b, c, d in params])
fig, ax1 = plt.subplots(figsize=(8, 4))
plt.xlabel('$b$')

ax1.set_ylabel('$\\mathfrak{r}(R^*)$')
ax1.yaxis.set_major_formatter(PercentFormatter(decimals=0))
Y = [100 * degree_of_restriction(result) for result in results]
ax1.plot(X, Y, label='Degree of restriction')

ax2 = ax1.twinx()
ax2.set_ylabel('# oracle calls')
Y = [result.info['number_of_oracle_calls'] for result in results]
ax2.plot(X, Y, color='g', label='Number of oracle calls')
ax2.set_ylim(bottom=0)

fig.legend()

# Save graph
fig.savefig(f'results/braess/degree.pdf', dpi=300, bbox_inches='tight')